In [1]:
import os
import time
import pandas as pd
from web3 import HTTPProvider, Web3
from pachira import *

In [2]:
chain = Net.POLYGON
platform = Platform.SUSHI
contract = JSONContract.UniswapV2Pair

json_rpc_url = os.environ.get(RPC.get_key(chain), RPC.get_rpc(chain))
web3 = Web3(HTTPProvider(json_rpc_url))
web3.middleware_onion.clear()

In [21]:
# Get contracts
abi_path = platform + '/' + contract + '.json'
Pair = ABILoad().get_contract(web3, abi_path)
swap_filt = Filter.create_filter(address=None, event_types=[Pair.events.Swap])  # Listen events from any smart contract

In [4]:
#ABILoading().get_abi_by_filename(abi_path)

In [22]:
reorg_mon = JSONRPCReorganizationMonitor(web3, check_depth=3)
reorg_mon.load_initial_block_headers(block_count=5)
processed_events = set()
latest_block = reorg_mon.get_last_block_live()
chain_reorg_resolution = reorg_mon.update_chain()
start, end = chain_reorg_resolution.get_read_range()
start = start - 25
print(f'latest_block {latest_block}, start {start}, end {end}')

latest_block 65326493, start 65326469, end 65326495


In [23]:
rEvents = ReadEvents().apply(web3, start_block=start, end_block=end, filter=swap_filt)

In [24]:
events = {}
evt: LogResult
for k, evt in enumerate(rEvents):
    # How to uniquely identify EVM logs
    key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]
    topics = evt["topics"]
    arguments = Conversion().decode_data(evt["data"])

    event = {}
    event['chain'] = chain
    event['contract'] = contract.lower()
    event['type'] = evt["event"].event_name.lower()
    event['platform'] = platform
    event['address'] = evt["address"]
    event['tx_hash'] = evt["transactionHash"]
    event['blk_num'] = evt["blockNumber"]
    event['timestamp'] = evt["timestamp"]
    event['details'] = {}
    event['details']['web3_type'] = evt["event"]
    event['details']['token0'] = Conversion().convert_uint256_hex_string_to_address(topics[1])
    event['details']['token1'] = Conversion().convert_uint256_hex_string_to_address(topics[2])
    event['details']['amount0In'] = Conversion().convert_int256_bytes_to_int(arguments[0]) 
    event['details']['amount1In'] = Conversion().convert_int256_bytes_to_int(arguments[1]) 
    event['details']['amount0Out'] = Conversion().convert_int256_bytes_to_int(arguments[2]) 
    event['details']['amount1Out'] = Conversion().convert_int256_bytes_to_int(arguments[3]) 
    
    events[k] = event
    if key not in processed_events:
        print(f"Swap at block:{evt['blockNumber']:,} tx:{evt['transactionHash']}")
        processed_events.add(key)
else:
    print(".")


Swap at block:65,326,469 tx:0x50275652c7db7dce959f95bd09b2592d88e62a96299023c7e536854e519808e2
Swap at block:65,326,469 tx:0x6648d9f2bc099dfa1468d7c259ac7c4ae4bf4d807f7851d5ec938f08bfb663dd
Swap at block:65,326,469 tx:0xc3afedc473654bb93499f7aba6dee38a8297d3307e3d5ba89b66d0a886ded24a
Swap at block:65,326,470 tx:0xb4ab0517276a4e82a550a7e4ee5c43203bbeabaf51529af9f969d71e9e843ce2
Swap at block:65,326,470 tx:0xb4ab0517276a4e82a550a7e4ee5c43203bbeabaf51529af9f969d71e9e843ce2
Swap at block:65,326,470 tx:0xb4ab0517276a4e82a550a7e4ee5c43203bbeabaf51529af9f969d71e9e843ce2
Swap at block:65,326,470 tx:0xf9df70722de8c9619e42ef94e8fd0bd758a757ac74d7b48a78c231d3d80192f9
Swap at block:65,326,470 tx:0xf9df70722de8c9619e42ef94e8fd0bd758a757ac74d7b48a78c231d3d80192f9
Swap at block:65,326,470 tx:0xf9df70722de8c9619e42ef94e8fd0bd758a757ac74d7b48a78c231d3d80192f9
Swap at block:65,326,471 tx:0x1012a72f85cfdd192ab801096d2dd892afefbabf3ddfd837352990ee60892e73
Swap at block:65,326,472 tx:0x421c189c3c49d36729cc

In [25]:
df_events = pd.DataFrame.from_dict(events, orient='index') 
df_events

,chain,contract,type,platform,address,tx_hash,blk_num,timestamp,details
0,polygon,uniswapv2pair,swap,sushi,0x232eb60e4ea68a164988329e16305bcacccf2a97,0x50275652c7db7dce959f95bd09b2592d88e62a962990...,65326469,1733866235,{'web3_type': <class 'web3._utils.datatypes.Sw...
1,polygon,uniswapv2pair,swap,sushi,0xccb9d2100037f1253e6c1682adf7dc9944498aff,0x6648d9f2bc099dfa1468d7c259ac7c4ae4bf4d807f78...,65326469,1733866235,{'web3_type': <class 'web3._utils.datatypes.Sw...
2,polygon,uniswapv2pair,swap,sushi,0xf6a637525402643b0654a54bead2cb9a83c8b498,0xc3afedc473654bb93499f7aba6dee38a8297d3307e3d...,65326469,1733866235,{'web3_type': <class 'web3._utils.datatypes.Sw...
3,polygon,uniswapv2pair,swap,sushi,0x882df4b0fb50a229c3b4124eb18c759911485bfb,0xb4ab0517276a4e82a550a7e4ee5c43203bbeabaf5152...,65326470,1733866237,{'web3_type': <class 'web3._utils.datatypes.Sw...
4,polygon,uniswapv2pair,swap,sushi,0x882df4b0fb50a229c3b4124eb18c759911485bfb,0xb4ab0517276a4e82a550a7e4ee5c43203bbeabaf5152...,65326470,1733866237,{'web3_type': <class 'web3._utils.datatypes.Sw...
...,...,...,...,...,...,...,...,...,...
95,polygon,uniswapv2pair,swap,sushi,0x30933831b71710d0551b518b74032242b2a71d2c,0x356bd1f81d936509f52b117aac8a3ff8332cad99df3d...,65326495,1733866289,{'web3_type': <class 'web3._utils.datatypes.Sw...
96,polygon,uniswapv2pair,swap,sushi,0xb5e1a07c9b6ab3bee8d9bf4066d324c5da89c07f,0xf67e37e8f62b2a80e8a96d0c198bc67660d3c17280eb...,65326495,1733866289,{'web3_type': <class 'web3._utils.datatypes.Sw...
97,polygon,uniswapv2pair,swap,sushi,0x50f724bf8270e77b5cc3e165ecfb244f32d802f6,0xa26148b6b5da8ddfe823bcae646459b56df1aa41febf...,65326495,1733866289,{'web3_type': <class 'web3._utils.datatypes.Sw...
98,polygon,uniswapv2pair,swap,sushi,0x809f7c69a7f24e0f74174c044770d5d5679722f3,0x75b046d18f1c979095b37b8c6202ec39ad010c968c71...,65326495,1733866289,{'web3_type': <class 'web3._utils.datatypes.Sw...


In [27]:
#events